# [Base Model Selection](https://learn.microsoft.com/azure/ai-foundry/concepts/evaluation-approach-gen-ai#base-model-selection)

The first stage of the AI lifecycle involves selecting an appropriate base model. Generative AI models vary widely in terms of capabilities, strengths, and limitations, so it's essential to identify which model best suits your specific use case. During base model evaluation, you "shop around" to compare different models by testing their outputs against a set of criteria relevant to your application.

You have three options to evaluate models:

1. [Use Azure AI Foundry Benchmarks](https://learn.microsoft.com/en-us/azure/ai-foundry/concepts/model-benchmarks) to compare models on their intrinsic capabilities.
1. [Use Manual Evaluations in the Portal](https://learn.microsoft.com/en-us/azure/ai-foundry/how-to/evaluate-prompts-playground) to run prompts on models and rate them.
1. [Evaluate Multiple Models using the SDK](https://learn.microsoft.com/en-us/azure/ai-foundry/how-to/evaluate-prompts-playground) code-first

**In this notebook, we explore a simplified version of option 3**

---


## Objective

You are beginning your AI application development journey - and you have two (or more) model options available to you. How do you pick the right one for your needs? In this tutorial we look at how you can evaluate _the same set of prompts_ against multiple model endpoints deployed in your Azure AI project.

This guide uses Python Class as an application target which is passed to Evaluate API provided by PromptFlow SDK to evaluate results generated by LLM models against provided prompts. 

This tutorial uses the following Azure AI services:

- [azure-ai-evaluation](https://learn.microsoft.com/en-us/azure/ai-studio/how-to/develop/evaluate-sdk)

## Time

You should expect to spend 30 minutes running this sample. 

## About this example

This example demonstrates evaluating model endpoints responses against provided prompts using azure-ai-evaluation

## Before you begin

### Installation

Install the following packages required to execute this notebook. 

In [1]:
!pip install azure-ai-evaluation --quiet

### Parameters and imports

In [2]:
from pprint import pprint

import pandas as pd
import random

## Target Application

We will use Evaluate API provided by Prompt Flow SDK. It requires a target Application or python Function, which handles a call to LLMs and retrieve responses. 

In the notebook, we will use an Application Target `ModelEndpoints` to get answers from multiple model endpoints against provided question aka prompts. 

This application target requires list of model endpoints and their authentication keys. For simplicity, we have provided them in the `env_var` variable which is passed into init() function of `ModelEndpoints`.

In [3]:
import os
endpoint = os.environ.get("AZURE_OPENAI_ENDPOINT")
api_version = os.environ.get("AZURE_OPENAI_API_VERSION")
key = os.environ.get("AZURE_OPENAI_KEY")

env_var = {
    "gpt4": {
        "endpoint": os.environ.get("AZURE_OPENAI_GPT4_EP"),
        "key": key,
    },
    "gpt-4o-mini": {
        "endpoint": os.environ.get("AZURE_OPENAI_GPT4OMINI_EP"),
        "key": key,
    },
}



Please provide Azure AI Project details so that traces and eval results are pushing in the project in Azure AI Studio.

In [4]:
import os

# Project Connection String
connection_string = os.environ.get("AZURE_AI_CONNECTION_STRING")

# Extract details
region_id, subscription_id, resource_group_name, project_name = connection_string.split(";")

# Populate it
azure_ai_project = {
    "subscription_id": subscription_id,
    "resource_group_name": resource_group_name,
    "project_name": project_name,
}
print(azure_ai_project)

{'subscription_id': '6415ebd4-1dd7-430f-bd4d-2f5e9419c1cd', 'resource_group_name': 'rg-nitya-lab334', 'project_name': 'ai-project-sn5orpaatr2i6'}


## Model Endpoints
The following code demonstrates how to call various model endpoints, and is configured based on `env_var` set above. For any model in `env_var`, if you do not have that model deployed in your AI project, please comment it out. If you have a model that you would like to test that does not correspond with one of the types seen below, please include that type in the `__call__` function and create a helper function to call the model's endpoint via REST. 

In [5]:
#!pygmentize model_endpoints.py

## Data

Following code reads Json file "data.jsonl" which contains inputs to the Application Target function. It provides question, context and ground truth on each line. 

In [6]:
df = pd.read_json("data.jsonl", lines=True)
print(df.head())

                                               query  \
0                     When was United Stated found ?   
1                     What is the capital of France?   
2                 Which tent is the most waterproof?   
3         Which camping table holds the most weight?   
4  What is the weight of the Adventure Dining Table?   

                                               truth  \
0                                               1776   
1                                              Paris   
2  The Alpine Explorer Tent has the highest rainf...   
3  The Adventure Dining Table has a higher weight...   
4           The Adventure Dining Table weighs 15 lbs   

                                              answer  
0                                               1600  
1                                              Paris  
2  Can you clarify what tents you are talking about?  
3                             Adventure Dining Table  
4                          It's a lot I can tell yo

## Configuration
To use Relevance and Cohenrence Evaluator, we will Azure Open AI model details as a Judge that can be passed as model config.

In [7]:
from azure.ai.evaluation import AzureOpenAIModelConfiguration

# Note: We are evaluating 2 models above - and we need a "LLM Judge" to evaluate them
#       Here we specify the judge model
model_config = AzureOpenAIModelConfiguration(
    azure_endpoint=os.environ.get("AZURE_OPENAI_ENDPOINT"),
    azure_deployment=os.environ.get("AZURE_OPENAI_DEPLOYMENT"),
    api_key=os.environ.get("AZURE_OPENAI_API_KEY"),
    api_version=os.environ.get("AZURE_OPENAI_API_VERSION"),
)
print(model_config)

{'azure_endpoint': 'https://aoai-sn5orpaatr2i6.openai.azure.com/', 'azure_deployment': 'gpt-4o-mini', 'api_key': '354f88b0334343e481afc7a6d5abe1d0', 'api_version': '2025-01-01-preview'}


## Run the evaluation

The Following code runs Evaluate API and uses Content Safety, Relevance and Coherence Evaluator to evaluate results from different models.

The following are the few parameters required by Evaluate API. 

+   Data file (Prompts): It represents data file 'data.jsonl' in JSON format. Each line contains question, context and ground truth for evaluators.     

+   Application Target: It is name of python class which can route the calls to specific model endpoints using model name in conditional logic.  

+   Model Name: It is an identifier of model so that custom code in the App Target class can identify the model type and call respective LLM model using endpoint URL and auth key.  

+   Evaluators: List of evaluators is provided, to evaluate given prompts (questions) as input and output (answers) from LLM models. 

In [9]:
import pathlib

from azure.ai.evaluation import evaluate
from azure.ai.evaluation import (
    RelevanceEvaluator,
)
from model_endpoints import ModelEndpoints

relevance_evaluator = RelevanceEvaluator(model_config)

models = [
    "gpt4",
    "gpt-4o-mini",
]

path = str(pathlib.Path(pathlib.Path.cwd())) + "/data.jsonl"
print(path)

for model in models:
    randomNum = random.randint(1111, 9999)
    results = evaluate(
        evaluation_name="Eval-Run-" + str(randomNum) + "-" + model.title(),
        data=path,
        target=ModelEndpoints(env_var, model),
        evaluators={
            "relevance": relevance_evaluator,
        },
        azure_ai_project=azure_ai_project,
        evaluator_config={
            "relevance": {
                "column_mapping": {
                    "response": "${target.response}",
                    "context": "${data.truth}",
                    "query": "${data.query}",
                },
            },
        },
    )

/workspaces/contoso-chat/docs/build25-lab334/notebooks/01-select-model/data.jsonl


[2025-05-08 17:24:51 +0000][promptflow._sdk._orchestrator.run_submitter][INFO] - Submitting run azure_ai_evaluation_evaluators_TARGET_20250508_172450_934483, log path: /home/vscode/.promptflow/.runs/azure_ai_evaluation_evaluators_TARGET_20250508_172450_934483/logs.txt


2025-05-08 17:24:54 +0000  112676 execution.bulk     INFO     Process 112707 terminated.
2025-05-08 17:24:54 +0000  112676 execution.bulk     WARNING  Process 112722 had been terminated.
2025-05-08 17:24:54 +0000  112676 execution.bulk     WARNING  Process 112716 had been terminated.
2025-05-08 17:24:54 +0000  112676 execution.bulk     WARNING  Process 112734 had been terminated.


[2025-05-08 17:24:55 +0000][promptflow._core.entry_meta_generator][WARNING] - Generate meta in current process and timeout won't take effect. Please handle timeout manually outside current process.
[2025-05-08 17:24:55 +0000][promptflow._sdk._orchestrator.run_submitter][INFO] - Submitting run azure_ai_evaluation_evaluators_relevance_20250508_172455_201279, log path: /home/vscode/.promptflow/.runs/azure_ai_evaluation_evaluators_relevance_20250508_172455_201279/logs.txt


2025-05-08 17:24:51 +0000  111621 execution.bulk     INFO     Current thread is not main thread, skip signal handler registration in BatchEngine.
2025-05-08 17:24:51 +0000  111621 execution          WARNING  Starting run without column mapping may lead to unexpected results. Please consult the following documentation for more information: https://aka.ms/pf/column-mapping
2025-05-08 17:24:51 +0000  111621 execution.bulk     INFO     Set process count to 4 by taking the minimum value among the factors of {'default_worker_count': 4, 'row_count': 5}.
2025-05-08 17:24:53 +0000  111621 execution.bulk     INFO     Process name(ForkProcess-8:4)-Process id(112734)-Line number(1) start execution.
2025-05-08 17:24:53 +0000  111621 execution.bulk     INFO     Process name(ForkProcess-8:1)-Process id(112707)-Line number(0) start execution.
2025-05-08 17:24:53 +0000  111621 execution.bulk     INFO     Process name(ForkProcess-8:3)-Process id(112722)-Line number(3) start execution.
2025-05-08 17:24:5

[2025-05-08 17:25:04 +0000][promptflow._sdk._orchestrator.run_submitter][INFO] - Submitting run azure_ai_evaluation_evaluators_TARGET_20250508_172504_837514, log path: /home/vscode/.promptflow/.runs/azure_ai_evaluation_evaluators_TARGET_20250508_172504_837514/logs.txt


2025-05-08 17:25:08 +0000  112968 execution.bulk     INFO     Process 113010 terminated.
2025-05-08 17:25:08 +0000  112968 execution.bulk     WARNING  Process 113007 had been terminated.
2025-05-08 17:25:08 +0000  112968 execution.bulk     WARNING  Process 113014 had been terminated.
2025-05-08 17:25:08 +0000  112968 execution.bulk     WARNING  Process 112995 had been terminated.


[2025-05-08 17:25:09 +0000][promptflow._core.entry_meta_generator][WARNING] - Generate meta in current process and timeout won't take effect. Please handle timeout manually outside current process.
[2025-05-08 17:25:09 +0000][promptflow._sdk._orchestrator.run_submitter][INFO] - Submitting run azure_ai_evaluation_evaluators_relevance_20250508_172509_055396, log path: /home/vscode/.promptflow/.runs/azure_ai_evaluation_evaluators_relevance_20250508_172509_055396/logs.txt


2025-05-08 17:25:04 +0000  111621 execution.bulk     INFO     Current thread is not main thread, skip signal handler registration in BatchEngine.
2025-05-08 17:25:04 +0000  111621 execution          WARNING  Starting run without column mapping may lead to unexpected results. Please consult the following documentation for more information: https://aka.ms/pf/column-mapping
2025-05-08 17:25:04 +0000  111621 execution.bulk     INFO     Set process count to 4 by taking the minimum value among the factors of {'default_worker_count': 4, 'row_count': 5}.
2025-05-08 17:25:06 +0000  111621 execution.bulk     INFO     Process name(ForkProcess-12:2)-Process id(113007)-Line number(1) start execution.
2025-05-08 17:25:06 +0000  111621 execution.bulk     INFO     Process name(ForkProcess-12:1)-Process id(112995)-Line number(0) start execution.
2025-05-08 17:25:06 +0000  111621 execution.bulk     INFO     Process name(ForkProcess-12:3)-Process id(113010)-Line number(2) start execution.
2025-05-08 17:2

View the results

In [ ]:
pprint(results)

In [ ]:
pd.DataFrame(results["rows"])